In [11]:
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
from numpy.testing import assert_array_equal, assert_array_almost_equal, assert_equal, assert_almost_equal
from pandas.testing import assert_frame_equal
from sklearn.datasets import make_classification

# SLIDE (2) Gini index

Необходимо получить функцию, которая находит правило, по которому нужно разбить массив признаков, по критерию gini.

In [7]:
import numpy as np

def get_gini_threshold(x: np.array, y:np.array):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [6]:
import numpy as np

def gini_impurity(y: np.ndarray):
    trgs = [0, 1]
    size = len(y)
    
    probs = np.zeros(len(trgs))
    for i, trg in enumerate(trgs):
        probs[i] = np.sum(y == trg) / size
        
    return 1 - np.sum(probs ** 2)

def get_gini_threshold(x, y):
    ### Sergey
    return best_threshold, best_score

# SLIDE (2) Энтропия

Необходимо получить функцию, которая находит правило, по которому нужно разбить массив признаков, по критерию entropy.

In [4]:
import numpy as np

def get_entropy_threshold(x: np.array, y:np.array):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    pass

In [8]:
def get_probs(y, trgs=[0, 1]):
    probs = np.zeros(len(trgs))
    size = len(y)
    assert size > 0
    for i, trg in enumerate(trgs):
        probs[i] = np.sum(y == trg) / size
    return probs

def entropy(y: np.ndarray):
    probs = get_probs(y)
    if np.any(probs == 0):
        return 0
    assert probs[0] > 0 and probs[1] > 0
    return -np.sum(probs * np.log2(probs))

def inform_gain(x, y, threshold):
    y_left = y[x <= threshold]
    y_right = y[x > threshold]
    if len(y_left) * len(y_right) == 0:
        return 0
    
    size = len(y)
    left_ratio = len(y_left) / size
    right_ratio = len(y_right) / size 
    
    return entropy(y) - left_ratio * entropy(y_left) - right_ratio * entropy(y_right)

def get_entropy_threshold(x, y):
    best_threshold = np.max(x)
    best_score = 0
    
    for threshold in np.unique(x):
        score = inform_gain(x, y, threshold)
        if score > best_score:
            best_threshold = threshold
            best_score = score
            
    return best_threshold, best_score

# SLIDE(2) Мой DecisionTree

Необходимо реализовать собственное небольшое дерево решений с единственным параметром: `max_length`.
Дерево должно быть бинарным (в ноде только одно простое правило). Класс листа выбирается самым популярным классом среди всех объектов обучения попавших в этот лист.

Алгоритм решения: Создаете объект Node и храните в нем признак и правило(`threshold`), по которому разбиваете. А также левого и правого ребенка. 

In [ ]:
from sklearn.base import BaseEstimator

class MyDecisionTreeClassifier(BaseEstimator):
    def __init__(self, max_depth=4):
        self.max_depth = max_depth
        self.tree = {}
    
    def _get_entropy_threshold(x, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return best_threshold, best_score
    
    def _find_best_split(self, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return best_feature, best_threshold, best_score
            
    def _build_tree(self, X, y):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return {'value': (split_feature, split_val), 
                'leaf' : False,
                'left' : left_tree, 
                'right': right_tree }
        
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        return self        
    
    def predict_proba(self, X_test):
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        pass
    
    def predict(self, X_test):
        proba = np.array(self.predict_proba(X_test))
        return np.argmax(proba, axis=1)

In [10]:
from sklearn.base import BaseEstimator, ClassifierMixin

class MyDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, max_depth=4):
        self.max_depth = max_depth
        self.tree = {}
        
    def find_best_split(self, X, y):
        best_feature = 0
        best_score = 0
        best_threshold = 0
        for feature_ind in range(X.shape[1]):
            X_feature = X[:, feature_ind]
            threshold, score = get_best_threshold(X_feature, y)
            if score > best_score:
                best_score = score
                best_feature = feature_ind
                best_threshold = threshold
        
        return best_feature, best_threshold, best_score
    
        
    def _build_tree(self, X, y, depth=0):
        if depth == self.max_depth:
            return {'leaf': True, 'proba': get_probs(y)}
        
        split_feature, split_val, score = self.find_best_split(X, y)
        
        if score == 0:
            return {'leaf': True, 'proba': get_probs(y)}
        
        left_inds  = X[:, split_feature] <= split_val
        right_inds = X[:, split_feature] >  split_val
        
        left_tree  = self._build_tree(X[left_inds],  y[left_inds],  depth + 1)
        right_tree = self._build_tree(X[right_inds], y[right_inds], depth + 1)
        
        return {'val': (split_feature, split_val), 'leaf': False,
                'left': left_tree, 'right': right_tree}
        
        
    def fit(self, X, y):
        self.tree = self._build_tree(X, y)
        return self        
        
    def _predict_proba_obj(self, obj):
        assert obj.ndim == 1     
        node = self.tree 
        while node['leaf'] == False:
            split_feature, split_val = node['val']
            if obj[split_feature] <= split_val:
                node = node['left']
            else:
                node = node['right']                
        return np.array(node['proba'])
    
    def predict_proba(self, X_test):
        return np.array([self._predict_proba_obj(row) for row in X_test])
    
    def predict(self, X_test):
        proba = np.array(self.predict_proba(X_test))
        return np.argmax(proba, axis=1)